In [27]:
import pandas as pd
import numpy as np
import re

def convert_date(date_str):
    # Extraire le fuseau horaire avec une expression régulière
    match = re.search(r'UTC([+-]\d)', date_str)
    if match:
        utc_offset = int(match.group(1))  # Récupérer le décalage horaire
    else:
        utc_offset = 0  # Valeur par défaut si aucun fuseau horaire n'est trouvé
    # Retirer le fuseau horaire et convertir en datetime
    date_without_tz = date_str.split(' UTC')[0]  # Retirer ' UTC+X'
    dt = pd.to_datetime(date_without_tz, format='%d/%m/%Y - %H:%M:%S')
    # Ajuster le fuseau horaire à UTC+4
    dt = dt + pd.Timedelta(hours=(4 - utc_offset))  # Ajustement basé sur l'offset
    return dt

# Fonction pour nettoyer les chaînes
def clean_number(number_str):
    # Utiliser une expression régulière pour supprimer le préfixe et le suffixe
    return re.sub(r'^\=\("\s*|\s*"\)$', '', number_str)

reunion_postal_codes = {
    "97400": "ST DENIS",
    "97410": "ST PIERRE",
    "97411": "BOIS DE NEFLES",
    "97412": "BRAS PANON",
    "97413": "CILAOS",
    "97414": "ENTRE DEUX",
    "97416": "LA CHALOUPE",
    "97417": "ST BERNARD",
    "97418": "LA PLAINE DES CAFRES",
    "97419": "LA POSSESSION",
    "97420": "LE PORT",
    "97421": "LA RIVIERE ST LOUIS",
    "97422": "LA SALINE",
    "97423": "LE GUILLAUME",
    "97424": "PITON ST LEU",
    "97425": "LES AVRIONS",
    "97426": "LES TROIS BASSINS",
    "97427": "L'ETANG SALE",
    "97428": "LA NOUVELLE",
    "97429": "PETITE ILE",
    "97430": "LE TAMPON",
    "97431": "LA PLAINE DES PALMISTES",
    "97432": "LA RAVINE DES CABRIS",
    "97433": "SALAZIE - HELL BOURG",
    "97434": "ST GILLES LES BAINS",
    "97435": "BERNICA",
    "97436": "ST LEU",
    "97437": "STE ANNE",
    "97438": "STE MARIE",
    "97439": "STE ROSE",
    "97440": "ST ANDRE",
    "97441": "STE SUZANNE",
    "97442": "BASSE VALLEE",
    "97450": "ST LOUIS",
    "97460": "ST PAUL",
    "97470": "ST BENOIT"
}

def replace_unknown_ville(row):
    if row['VILLE'] == 'ville inconnue':
        return reunion_postal_codes.get(row['CODE POSTAL'], 'ville inconnue')  # Remplace par la ville correspondante ou garde 'ville inconnue'
    return row['VILLE']


def preprocess_data(file1):
    df = pd.read_csv(file1, sep=';', encoding='latin1', dtype={"CIBLE": str, "CORRESPONDANT": str, "DUREE": str, "IMSI": str, "IMEI": str, "CODE POSTAL": str,  "X": str, "Y": str, "VILLE": str, "ADRESSE2": str, "ADRESSE3": str, "ADRESSE4": str, "ADRESSE5": str})
    # Appliquer la fonction pour convertir les dates
    df['converted_date'] = df['DATE'].apply(convert_date)
    # Extraire l'année, le mois et le jour de la semaine
    df['Années'] = df['converted_date'].dt.year
    df['Mois'] = df['converted_date'].dt.month
    df['Jour de la semaine'] = df['converted_date'].dt.day_name()
    # Mapper les jours de la semaine en français
    jours_semaine_fr = {
        'Monday': 'Lundi',
        'Tuesday': 'Mardi',
        'Wednesday': 'Mercredi',
        'Thursday': 'Jeudi',
        'Friday': 'Vendredi',
        'Saturday': 'Samedi',
        'Sunday': 'Dimanche'
    }
    # Mapper les mois en français
    mois_fr = {
        1: 'Janvier',
        2: 'Février',
        3: 'Mars',
        4: 'Avril',
        5: 'Mai',
        6: 'Juin',
        7: 'Juillet',
        8: 'Août',
        9: 'Septembre',
        10: 'Octobre',
        11: 'Novembre',
        12: 'Décembre'
    }
    # Remplacer les numéros de mois par leur équivalent en français
    df['Mois'] = df['Mois'].map(mois_fr)
    # Remplacer les noms des jours par leur équivalent en français
    df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)
    df["CORRESPONDANT"] = df["CORRESPONDANT"].fillna("Data")
    df['CORRESPONDANT'] = df['CORRESPONDANT'].str.split(',').str[0]
    df['CORRESPONDANT'] = df['CORRESPONDANT'].apply(clean_number)
    df['CORRESPONDANT'] = df['CORRESPONDANT'].replace(r'^0692', '262692', regex=True)
    df['CORRESPONDANT'] = df['CORRESPONDANT'].replace(r'^0693', '262693', regex=True)
    df['CORRESPONDANT'] = df['CORRESPONDANT'].replace(r'^06', '336', regex=True)
    df['CORRESPONDANT'] = df['CORRESPONDANT'].replace(r'^07', '337', regex=True)
    df['CORRESPONDANT'] = df['CORRESPONDANT'].replace(r'^02', '2622', regex=True)
    df["DUREE"] = df["DUREE"].fillna("0")
    df['IMEI'] = df['IMEI'].fillna("INDETERMINE")
    df["IMEI"] = df["IMEI"].apply(clean_number)
    df['IMSI'] = df['IMSI'].apply(clean_number)
    df['CIBLE'] = df['CIBLE'].apply(clean_number)
    df['CIBLE'] = df['CIBLE'].replace(r'^0692', '262692', regex=True)
    df['CIBLE'] = df['CIBLE'].replace(r'^0693', '262693', regex=True)
    df['CIBLE'] = df['CIBLE'].replace(r'^06', '336', regex=True)
    df['VILLE'] = df.apply(replace_unknown_ville, axis=1)
    df['VILLE'] = df['VILLE'].str.upper()
    df['VILLE']= df['VILLE'].str.replace("-", " ")
    df['VILLE']= df['VILLE'].str.replace("SAINT", "ST")
    df['VILLE']= df['VILLE'].str.replace("SAINTE", "STE")
    df['VILLE']= df['VILLE'].str.replace("L'", "")
    df['VILLE'] = df['VILLE'].str.replace("É", "E", regex=False)
    df["Adresse"] = df["ADRESSE2"] + " " + df["CODE POSTAL"] + " " + df["VILLE"]
    df.fillna({'Adresse': 'INDETERMINE', 'VILLE' : 'INDETERMINE', 'CODE POSTAL' : 'INDETERMINE' }, inplace=True)
    df['Adresse'] = df['Adresse'].str.upper()
    df['Adresse']= df['Adresse'].str.replace(r'\s+', ' ', regex=True)
    df['Adresse'] = df['Adresse'].str.strip() # Supprimer les espaces inutiles
    deleted_columns = ['DATE', 'TYPE CORRESPONDANT', 'COMP.', 'EFFICACITE' , 'CELLID', 'ADRESSE IP VO WIFI', 'PORT SOURCE VO WIFI', 'ADRESSE2','ADRESSE3','ADRESSE4', 'ADRESSE5', 'PAYS', 'TYPE-COORD', 'CODE POSTAL']
    df.drop(columns=deleted_columns, inplace=True, errors='ignore')
    rename_dict = {"TYPE": "Type d'appel", "CORRESPONDANT": "Correspondant", "CIBLE": "Abonné", "DIRECTION": "Direction", "DUREE": "Durée", "VILLE": "Ville", "X": "Latitude", "Y": "Longitude", "converted_date": "Date"
    }
    # Renommer uniquement les colonnes présentes dans le DataFrame
    df.rename(columns={k: v for k, v in rename_dict.items() if k in df.columns}, inplace=True)
    return df

In [28]:
df = preprocess_data("/home/antoine/telephony_insight_project/streamlit/app/source/models_files/TCOI_MT24.csv")

In [40]:
df.head(50)

,Type d'appel,Abonné,Correspondant,Direction,Durée,IMSI,IMEI,Ville,Latitude,Longitude,Date,Années,Mois,Jour de la semaine,Adresse
0,TELEPHONIE,262693856874,262262352253,sortant,63,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024-06-14 09:04:44,2024,Juin,Vendredi,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
1,DATA_PAQUET,262693856874,Data,sortant,68,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024-06-14 09:04:59,2024,Juin,Vendredi,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
2,DATA_PAQUET,262693856874,Data,sortant,68,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024-06-14 09:04:59,2024,Juin,Vendredi,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
3,TELEPHONIE,262693856874,262692109747,entrant,38,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024-06-14 11:18:51,2024,Juin,Vendredi,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU
4,DATA_PAQUET,262693856874,Data,sortant,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024-06-14 11:18:53,2024,Juin,Vendredi,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU
5,DATA_PAQUET,262693856874,Data,sortant,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024-06-14 11:18:53,2024,Juin,Vendredi,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU
6,DATA_PAQUET,262693856874,Data,sortant,36,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024-06-14 11:19:09,2024,Juin,Vendredi,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
7,DATA_PAQUET,262693856874,Data,sortant,36,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024-06-14 11:19:09,2024,Juin,Vendredi,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
8,TELEPHONIE,262693856874,262262352253,entrant,53,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024-06-14 13:51:33,2024,Juin,Vendredi,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU
9,DATA_PAQUET,262693856874,Data,sortant,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024-06-14 13:51:34,2024,Juin,Vendredi,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU


In [30]:
df.Adresse.unique()

array(['62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU',
       '215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU', 'INDETERMINE',
       'RUELLE DES ZATTES 97416 ST LEU',
       '21 AV DES ARTISANS 97416 ST LEU',
       'LA SALINE LES HAUTS 97460 ST PAUL',
       '142 RUE LECONTE DELISLE 97450 ST LOUIS',
       'ROUTE DU MAÏDO 97423 ST PAUL',
       'RUE AUGUST VINSON STADE 97460 ST PAUL',
       '11 CHEMIN LEBOT, BOIS DE NA?FLES ST PAUL 97411 ST PAUL',
       '194, CHEMIN SUMMER, CHEMIN PAVE LOUGNON 97460 ST PAUL',
       'TERRAIN CADASTRÉ BX 82 - LIEU DIT BELLEMÈNE - 45 97460 ST PAUL',
       '149 RUE DES POINSETIAS, PK 17 97430 LE TAMPON',
       '13 CHEMIN NEUF- LA PLAINE ST PAUL 97460 ST PAUL',
       '35 37 RUE HO CHI MIN 97419 LA POSSESSION',
       '100 CHEMIN DES ROSES VILLÈLE 97435 ST GILLES LES HAUTS',
       'NC 97410 ST PIERRE',
       'CHEMIN DU PÈRE TABAILLET LIEU-DIT LA POINTE 97416 PITON ST LEU',
       '24 CHEMIN DES GIROFLÉS - COLIMAÇONS 97419 CHALOUPE ST LEU',
       'ROUTE DE

In [31]:
import plotly.graph_objects as go
import plotly.express as px

# Créer une carte des adresses pour l'opérateur TCOI.


def carto_adresse_tcoi(df):
    try:
        fig = px.scatter_map(df,
                            lat="Latitude", lon="Longitude",
                            color="Pourcentage", size="Nombre de déclenchement",
                            hover_name="Adresse",
                            size_max=60, zoom=10,
                            color_continuous_scale=px.colors.sequential.Bluered,
                            map_style="carto-positron")

        return fig
    except Exception as e:
        print(f"Erreur lors de la création de la carte des adresses: {e}")
        return go.Figure()

In [32]:
def adresse_count(df):
    try:
       adresse_counts = df['Adresse'].value_counts().reset_index()
       adresse_counts.columns = ['Adresse', 'Nombre de déclenchement']
       adresse_counts.dropna(axis=0, inplace=True)  # Supprimer les lignes sans adresse
       total_contacts = adresse_counts['Nombre de déclenchement'].sum()
       adresse_counts['Pourcentage'] = ((adresse_counts['Nombre de déclenchement'] / total_contacts) * 100).round(1)
       adresse_counts.sort_values(by='Nombre de déclenchement', ascending=False, inplace=True)
       return adresse_counts
    except Exception as e:
       print(f"Erreur lors du comptage des adresses: {e}")
       return pd.DataFrame()

In [33]:
adresse_co = adresse_count(df)
adresse_co.head(50)

,Adresse,Nombre de déclenchement,Pourcentage
0,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU,1435,31.3
1,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU,791,17.2
2,INDETERMINE,581,12.7
3,67 RUE HENRI CORNU 97460 ST PAUL,458,10.0
4,633 CHEMIN CENT GAULETTES 97440 ST ANDRE,183,4.0
5,21 AV DES ARTISANS 97416 ST LEU,148,3.2
6,LA SALINE LES HAUTS 97460 ST PAUL,146,3.2
7,24 CHEMIN DES GIROFLÉS - COLIMAÇONS 97419 CHAL...,143,3.1
8,RUELLE DES ZATTES 97416 ST LEU,94,2.0
9,13 CHEMIN MARGUERITES 97426 TROIS BASSINS,79,1.7


In [34]:

new_df = adresse_co.merge(df, on='Adresse', how='left')

In [35]:
carto_adresse_tcoi(new_df)

Erreur lors de la création de la carte des adresses: can only concatenate str (not "int") to str


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4587 entries, 0 to 4586
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Adresse                  4587 non-null   object        
 1   Nombre de déclenchement  4587 non-null   int64         
 2   Pourcentage              4587 non-null   float64       
 3   Type d'appel             4587 non-null   object        
 4   Abonné                   4587 non-null   object        
 5   Correspondant            4587 non-null   object        
 6   Direction                4587 non-null   object        
 7   Durée                    4587 non-null   object        
 8   IMSI                     4587 non-null   object        
 9   IMEI                     4587 non-null   object        
 10  Ville                    4587 non-null   object        
 11  Latitude                 4006 non-null   object        
 12  Longitude                4006 non-

In [36]:
# Convertir les colonnes en types appropriés si nécessaire
new_df['Latitude'] = pd.to_numeric(new_df['Latitude'], errors='coerce')
new_df['Longitude'] = pd.to_numeric(new_df['Longitude'], errors='coerce')
new_df['Pourcentage'] = pd.to_numeric(new_df['Pourcentage'], errors='coerce')
new_df['Nombre de déclenchement'] = pd.to_numeric(new_df['Nombre de déclenchement'], errors='coerce')

# Supprimer les lignes avec des valeurs manquantes dans les colonnes critiques
new_df.dropna(subset=['Latitude', 'Longitude', 'Pourcentage', 'Nombre de déclenchement'], inplace=True)

In [37]:
carto_adresse_tcoi(new_df)

In [38]:
def carto_adresse_tcoi(df):
    try:
        fig = px.scatter_mapbox(
            df,
            lat="Latitude",
            lon="Longitude",
            color="Pourcentage",
            size="Nombre de déclenchement",
            hover_name="Adresse",
            size_max=60,
            zoom=10,
            color_continuous_scale=px.colors.sequential.Bluered,
            mapbox_style="carto-positron"
        )
        return fig
    except Exception as e:
        st.error(f"Erreur lors de la création de la carte des adresses : {e}")
        return go.Figure()

In [39]:
carto_adresse_tcoi(new_df)